In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
from pprint import pprint

# Resource File (CSV)
five_cities = pd.read_csv("output_data/restaurants_category_10_five_cities.csv")
five_cities.head()

,Unnamed: 0,Restaurant_id,Name,Locality,Address,City,Latitude,Longitude,Zip Code,Cuisines,Price Range,Average Cost for two,User Rating,Rating Text,Votes,all_reviews_count
0,0,15547004,Restaurant Hubert,CBD,"15 Bligh Street, CBD, Sydney",Sydney,-33.865348,151.210624,2000.0,"French, European",4,150,4.9,Excellent,589,224
1,1,16558798,Quay,Circular Quay,"Upper Level, Overseas Passenger Terminal 5 Hic...",Sydney,-33.858029,151.209970,2000.0,Modern Australian,4,500,4.9,Excellent,1366,454
2,2,16559171,Tetsuya's,CBD,"529 Kent Street, CBD, Sydney",Sydney,-33.875143,151.204932,NaN,Japanese,4,440,4.9,Excellent,1234,329
3,3,16569454,LuMi Bar & Dining,Pyrmont,"56 Pirrama Road, \tPyrmont, Pyrmont, Sydney",Sydney,-33.867137,151.197517,2009.0,"Italian, Japanese",4,190,4.9,Excellent,452,196
4,4,15545439,Manpuku,Chatswood,"226 Victoria Avenue, Chatswood, Sydney",Sydney,-33.794417,151.189542,2067.0,"Japanese, Ramen",2,40,4.9,Excellent,486,190


In [2]:
# Replace all NaN values with 0 
five_cities_clean = five_cities.fillna(0)
#Select rows with no zip code
missing_zipcode = five_cities_clean.loc[five_cities_clean["Zip Code"]== 0]
missing_zipcode.count()

Unnamed: 0              22
Restaurant_id           22
Name                    22
Locality                22
Address                 22
City                    22
Latitude                22
Longitude               22
Zip Code                22
Cuisines                22
Price Range             22
Average Cost for two    22
User Rating             22
Rating Text             22
Votes                   22
all_reviews_count       22
dtype: int64

In [3]:
#Replacing the empty zipcode by a central zipcode
missing_zipcode.loc[missing_zipcode['City'] == "Sydney", 'Zip Code'] = 2000
missing_zipcode.loc[missing_zipcode['City'] == "Melbourne", 'Zip Code'] = 3000
missing_zipcode.loc[missing_zipcode['City'] == "Brisbane", 'Zip Code'] = 4000
missing_zipcode.loc[missing_zipcode['City'] == "Adelaide", 'Zip Code'] = 5000
missing_zipcode.loc[missing_zipcode['City'] == "Perth", 'Zip Code'] = 6000
missing_zipcode

C:\Users\Babette\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Unnamed: 0,Restaurant_id,Name,Locality,Address,City,Latitude,Longitude,Zip Code,Cuisines,Price Range,Average Cost for two,User Rating,Rating Text,Votes,all_reviews_count
2,2,16559171,Tetsuya's,CBD,"529 Kent Street, CBD, Sydney",Sydney,-33.875143,151.204932,2000.0,Japanese,4,440,4.9,Excellent,1234,329
10,10,17744148,Ho Jiak Haymarket 好吃禧市,Haymarket,"92 Hay Street, Chinatown, Haymarket, Sydney",Sydney,-33.879366,151.204054,2000.0,"Malaysian, Asian, Chinese, Asian Fusion, Singa...",3,70,4.8,Excellent,553,218
25,25,16562375,Hugos Manly,"Manly Wharf, Manly","Shop 1, Manly Wharf, 21 East Esplanade, Manly,...",Sydney,-33.799878,151.283890,2000.0,"Modern Australian, Tapas",4,130,4.7,Excellent,467,157
70,70,16559765,Yiamas Greek Taverna,St. Peters,"5 Princes Hwy, St. Peters, Sydney",Sydney,-33.908910,151.180881,2000.0,Greek,2,55,4.6,Excellent,101,48
80,80,17743923,Juan Bowl & Tea,Redfern,"Shop 5 94A Pitt Street, Redfern, Sydney",Sydney,-33.893095,151.203899,2000.0,"Japanese, Tea",3,80,4.5,Excellent,341,117
95,95,17744545,Gogyo,Surry Hills,"52-54 Albion Street, Surry Hills, Sydney",Sydney,-33.882268,151.210435,2000.0,"Japanese, Ramen",2,50,4.5,Excellent,342,116
99,99,16542060,Chat Thai - Westfield Pitt Street Mall,"Westfield Pitt Street Mall, CBD","Level 6, Westfield Pitt Street Mall, 188 Pitt ...",Sydney,-33.870114,151.209347,2000.0,"Thai, Salad",2,55,4.5,Excellent,730,278
103,3,17881527,Dexter,Preston,"456 High Street, Preston, Melbourne",Melbourne,-37.736196,145.004456,3000.0,"American, BBQ",4,110,4.9,Excellent,1473,682
139,39,16577899,Myong Tofu,Flemington,"314 Racecourse Road, Flemington, Melbourne",Melbourne,-37.788108,144.930240,3000.0,"Malaysian, Chinese",3,70,4.7,Excellent,694,312
143,43,16573158,Fresh Chilli Thai Restaurant,Laverton,"Shop 11, 5 Aviation Road, Laverton, Melbourne",Melbourne,-37.867439,144.761923,3000.0,"Asian, Thai",3,60,4.7,Excellent,464,193


In [4]:
#Drop the rows of missing postcodes from five_cities_clean df
new_df = five_cities_clean.loc[five_cities_clean["Zip Code"] != 0]
new_df.count()

Unnamed: 0              478
Restaurant_id           478
Name                    478
Locality                478
Address                 478
City                    478
Latitude                478
Longitude               478
Zip Code                478
Cuisines                478
Price Range             478
Average Cost for two    478
User Rating             478
Rating Text             478
Votes                   478
all_reviews_count       478
dtype: int64

In [5]:
#add the 22 rows with replaced postcodes
frames = [new_df, missing_zipcode]
all_zipcodes = pd.concat(frames)
all_zipcodes.count()

Unnamed: 0              500
Restaurant_id           500
Name                    500
Locality                500
Address                 500
City                    500
Latitude                500
Longitude               500
Zip Code                500
Cuisines                500
Price Range             500
Average Cost for two    500
User Rating             500
Rating Text             500
Votes                   500
all_reviews_count       500
dtype: int64

In [6]:
check = all_zipcodes.sort_values(by="Zip Code")
check.head(23)

,Unnamed: 0,Restaurant_id,Name,Locality,Address,City,Latitude,Longitude,Zip Code,Cuisines,Price Range,Average Cost for two,User Rating,Rating Text,Votes,all_reviews_count
0,0,15547004,Restaurant Hubert,CBD,"15 Bligh Street, CBD, Sydney",Sydney,-33.865348,151.210624,2000.0,"French, European",4,150,4.9,Excellent,589,224
36,36,16557481,Cafe Sydney,CBD,"Level 5, Customs House, 31 Alfred Street, CBD,...",Sydney,-33.862423,151.210899,2000.0,"Seafood, Modern Australian",4,160,4.6,Excellent,1387,370
38,38,16558709,The Restaurant Pendolino,"The Strand Arcade, CBD","Shop 100-102, Level 2, The Strand Arcade, 412-...",Sydney,-33.869004,151.207235,2000.0,Italian,4,250,4.6,Excellent,783,192
39,39,16569275,Do Dee Paidang Thai,Haymarket,"Shop 9, 37 Ultimo Road, Haymarket, Sydney",Sydney,-33.880568,151.203137,2000.0,"Thai, Asian",2,35,4.6,Excellent,406,167
44,44,16557297,Bentley Restaurant and Bar - Radisson Blu Plaz...,Radisson Blu Plaza Hotel CBD,"Radisson Blue Plaza Hotel, 27 O'Connell Street...",Sydney,-33.865651,151.208893,2000.0,Modern Australian,4,170,4.6,Excellent,403,178
45,45,16714055,Yang San Park,Haymarket,"21/1 Dixon Street, Haymarket",Sydney,-33.877494,151.203858,2000.0,Korean BBQ,2,40,4.6,Excellent,328,113
47,47,16570503,Nel. Restaurant,CBD,"75 Wentworth Avenue, CBD, Sydney",Sydney,-33.879386,151.209543,2000.0,"British, Australian",4,220,4.6,Excellent,200,95
51,51,16567175,The Lobo Plantation,CBD,"Basement, 209 Clarence Street, CBD, Sydney",Sydney,-33.870016,151.205390,2000.0,"Bar Food, Latin American",3,80,4.6,Excellent,214,69
80,80,17743923,Juan Bowl & Tea,Redfern,"Shop 5 94A Pitt Street, Redfern, Sydney",Sydney,-33.893095,151.203899,2000.0,"Japanese, Tea",3,80,4.5,Excellent,341,117
54,54,16563274,Grandma's Bar,CBD,"Basement, 275 Clarence Street, \tCBD",Sydney,-33.872182,151.205526,2000.0,"Bar Food, Sandwich",3,90,4.6,Excellent,268,57
